# The Rosenbluth-Fokker-Planck equation

- Import all relevant modules

In [1]:
import torch
from pyapes.geometry import Cylinder
from pyapes.mesh import Mesh
from pyrfp.simulators.rfp import RFP_RZ
from pyrfp.training_data import RosenbluthPotentials_RZ
from pymytools.constants import PI


- Set some parameters to be used later

In [2]:
device="cpu"
grids = [[32, 64], [64, 128], [128, 256], [256, 512]]

- Solver accuracy at different grids

In [3]:
for g in grids:
    mesh = Mesh(Cylinder[0:5, -5:5], None, g, device=device)
    dist = 1.0 / (2.0 * PI) ** 1.5 * torch.exp(-0.5 * (mesh.R**2 + mesh.Z**2))

    density = torch.sum(dist * 2.0 * PI * mesh.dx[0] * mesh.dx[1] * mesh.R)
    dist /= density
    
    rp_rz = RosenbluthPotentials_RZ(mesh, solver_config={
            "method": "bicgstab",
            "tol": 1e-8,
            "max_it": 1000,
            "report": False,
        })
    
    res_sol = rp_rz.from_pdf(dist, disp=False)
    res_analytic = rp_rz.from_analytic(dist, disp=False)
    
    assert res_sol["timer"] is not None
    assert res_analytic["timer"] is not None
    
    assert res_sol["pots"] is not None
    assert res_analytic["pots"] is not None
    
    print(f"Grid: {g}")
    res_sol["timer"].display()
    
    rp_rz.timer.reset()



Grid: [32, 64]


             Timer Report             
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃  Name  ┃ Elapsed time [s] ┃ #Calls ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│  H_bc  │     0.01489      │   1    │
│ H_pot  │     0.05342      │   1    │
│  G_bc  │     0.01074      │   1    │
│ G_pot  │     0.04690      │   1    │
│ aH_pot │     0.08827      │   1    │
│ aG_pot │     0.06233      │   1    │
└────────┴──────────────────┴────────┘

Grid: [64, 128]


             Timer Report             
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃  Name  ┃ Elapsed time [s] ┃ #Calls ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│  H_bc  │     0.06543      │   1    │
│ H_pot  │     0.23041      │   1    │
│  G_bc  │     0.06327      │   1    │
│ G_pot  │     0.22706      │   1    │
│ aH_pot │     1.18830      │   1    │
│ aG_pot │     1.15290      │   1    │
└────────┴──────────────────┴────────┘

: 

: 

- Computational cost with different grid sized

In [2]:

for g in grids:
    mesh = Mesh(Cylinder[0:5, -5:5], None, g, device=device)

    dist = 1.0 / (2.0 * PI) ** 1.5 * torch.exp(-0.5 * (mesh.R**2 + mesh.Z**2))

    density = torch.sum(dist * 2.0 * PI * mesh.dx[0] * mesh.dx[1] * mesh.R)
    dist /= density

    rfp_rz = RFP_RZ(mesh, dist, 0.0001, 10)
    rfp_rz.run(no_update=True)
    print(f"Tested grid: {g}")
    rfp_rz.timer.display()
    rfp_rz.timer.reset()

Tested grid: [32, 64]


            Timer Report             
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Name  ┃ Elapsed time [s] ┃ #Calls ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ H_bc  │     0.08918      │   10   │
│ H_sol │     0.37887      │   10   │
│ G_bc  │     0.08756      │   10   │
│ G_sol │     0.39268      │   10   │
└───────┴──────────────────┴────────┘

Tested grid: [64, 128]


            Timer Report             
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Name  ┃ Elapsed time [s] ┃ #Calls ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ H_bc  │     0.59745      │   10   │
│ H_sol │     1.68591      │   10   │
│ G_bc  │     0.59088      │   10   │
│ G_sol │     1.90611      │   10   │
└───────┴──────────────────┴────────┘

Tested grid: [128, 256]


            Timer Report             
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Name  ┃ Elapsed time [s] ┃ #Calls ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ H_bc  │     4.39087      │   10   │
│ H_sol │     10.43475     │   10   │
│ G_bc  │     4.17052      │   10   │
│ G_sol │     10.84780     │   10   │
└───────┴──────────────────┴────────┘

Tested grid: [256, 512]


            Timer Report             
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Name  ┃ Elapsed time [s] ┃ #Calls ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ H_bc  │     57.30470     │   10   │
│ H_sol │     74.41498     │   10   │
│ G_bc  │     55.42355     │   10   │
│ G_sol │     71.48135     │   10   │
└───────┴──────────────────┴────────┘